In [154]:
import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup
import json
import time
from tqdm.notebook import tqdm

In [2]:
URL = "https://www.metoffice.gov.uk/research/climate/maps-and-data/historic-station-data"
r = requests.get(URL,'xml')
  
soup = BeautifulSoup(r.content, 'xml') 
xml = soup.prettify()

In [1]:
# print(xml)

In [38]:
station_list = [i.text for i in soup.find_all('td')[0::4]]
station_coordinates = [i.text for i in soup.find_all('td')[1::4]]
station_start_year = [i.text for i in soup.find_all('td')[2::4]]
station_link = [i.a['href'] for i in soup.find_all('td')[3::4]]

In [61]:
df = pd.DataFrame({'station_list': station_list,
              'station_coordinates': station_coordinates,
              'station_start_year': station_start_year,
              'station_link': station_link,
             })

In [64]:
URL = df['station_link'][0]
r = requests.get(URL)

In [181]:
dfs = []
for i, j in tqdm(zip(df['station_link'], df['station_list'])):
    time.sleep(np.random.randint(3) + np.random.random())
    r = requests.get(i)
    some_list = [[i for i in f.split(' ') if i not in ['']] for f in \
                 r.text.split('              degC    degC    days      mm   hours\r\n')[1].split('\r\n')]
    sdf = pd.DataFrame(some_list)
    if sdf.shape[1]==8:
        sdf = pd.DataFrame(some_list, columns=['year', 'month', 'tmax(degC)', 'tmin(degC)', 'air_frost(days)', \
                                               'rainfall(mm)', 'sun(hours)', 'provisional'])
    elif sdf.shape[1]==7:
        sdf = pd.DataFrame(some_list, columns=['year', 'month', 'tmax(degC)', 'tmin(degC)', 'air_frost(days)', \
                                           'rainfall(mm)', 'sun(hours)'])
    else:
        sdf = pd.DataFrame(some_list, columns=['year', 'month', 'tmax(degC)', 'tmin(degC)', 'air_frost(days)', \
                                           'rainfall(mm)', 'sun(hours)', 'provisional', 'add1', 'add2'])    
    sdf['sunshine_sensor'] = sdf['sun(hours)'].str.contains(r'\*|#').replace({True: 'automatic Kipp & Zonen sensor', \
                                                                          False: 'Campbell Stokes recorder'})
    sdf['sun(hours)'] = sdf['sun(hours)'].str.replace(r'\*|#', '')
    try:
        sdf['provisional'] = sdf['provisional'].replace({None: 0, 'Provisional': 1})
    except:
        pass
    sdf = sdf.replace('---', np.nan)
    sdf.insert(0, 'station_location', j)
    dfs.append(sdf)

0it [00:00, ?it/s]

C:\Users\rockm\AppData\Local\Temp/ipykernel_2168/2388208685.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  sdf['sun(hours)'] = sdf['sun(hours)'].str.replace(r'\*|#', '')


In [180]:
# df2 = dfs.copy()

In [184]:
oof = pd.concat(dfs, ignore_index=True)

In [186]:
oof.to_excel('met_station_data.xlsx', index=False)

In [47]:
# URL = "https://findthatpostcode.uk/points/52.13914%2C-4.56999.json"
# r = requests.get(URL)
# soup = BeautifulSoup(r.content, 'html')
# site_json=json.loads(soup.text)
# site_json

In [188]:
%pip install netCDF4 -q

Note: you may need to restart the kernel to use updated packages.


In [340]:
import netCDF4 as nc
fn = r"C:\Users\rockm\Downloads\sfcWind_hadukgrid_uk_1km_ann_202001-202012.nc"
ds = nc.Dataset(fn)

In [191]:
ds

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    comment: annual resolution gridded climate observations
    creation_date: 2021-07-12T19:59:14
    frequency: ann
    institution: Met Office
    references: doi: 10.1002/joc.1161
    short_name: annual_windspeed
    source: HadUK-Grid_v1.0.3.0
    title: Gridded surface climate observations data for the UK
    version: v20210712
    Conventions: CF-1.7
    dimensions(sizes): time(1), projection_y_coordinate(1450), projection_x_coordinate(900), bnds(2)
    variables(dimensions): float64 sfcWind(time, projection_y_coordinate, projection_x_coordinate), int32 transverse_mercator(), float64 time(time), float64 time_bnds(time, bnds), float64 projection_y_coordinate(projection_y_coordinate), float64 projection_y_coordinate_bnds(projection_y_coordinate, bnds), float64 projection_x_coordinate(projection_x_coordinate), float64 projection_x_coordinate_bnds(projection_x_coordinate, bnds), float64 latitude(pr

In [329]:
ds.variables

{'sfcWind': <class 'netCDF4._netCDF4.Variable'>
 float64 sfcWind(time, projection_y_coordinate, projection_x_coordinate)
     _FillValue: 1e+20
     standard_name: wind_speed
     long_name: Wind speed at 10m
     units: m s-1
     description: Wind speed
     label_units: m s-1
     level: 10m
     plot_label: Wind speed at 10m (m s-1)
     cell_methods: time: mean time: mean
     grid_mapping: transverse_mercator
     coordinates: latitude longitude
 unlimited dimensions: 
 current shape = (1, 1450, 900)
 filling on,
 'transverse_mercator': <class 'netCDF4._netCDF4.Variable'>
 int32 transverse_mercator()
     grid_mapping_name: transverse_mercator
     longitude_of_prime_meridian: 0.0
     semi_major_axis: 6377563.396
     semi_minor_axis: 6356256.909
     longitude_of_central_meridian: -2.0
     latitude_of_projection_origin: 49.0
     false_easting: 400000.0
     false_northing: -100000.0
     scale_factor_at_central_meridian: 0.9996012717
 unlimited dimensions: 
 current shape = (

In [382]:
ds.variables['sfcWind'][0, 400, 405]

masked_array(data=6.85199285,
             mask=False,
       fill_value=1e+20)

In [383]:
ds.variables['projection_y_coordinate'][:][400], ds.variables['projection_x_coordinate'][:][405]

(200500.0, 205500.0)

In [394]:
print(ds.variables['latitude'][:][400, 405], ds.variables['longitude'][:][400, 405])
print(ds.variables['latitude'][:][401, 405], ds.variables['longitude'][:][401, 405])

51.669090121947875 -4.813029317373068
51.6780715139834 -4.81358620267212


In [405]:
a = ds.variables['longitude'][:][:, 405]
np.all(np.diff(a) >= 0)

False

In [404]:
ds.variables['projection_y_coordinate'][:][-1], ds.variables['projection_x_coordinate'][:][-1]

(1249500.0, 699500.0)

In [230]:
pd.Timestamp('1800-01-01') + pd.tseries.offsets.DateOffset(hours=1932864)

Timestamp('2020-07-02 00:00:00')

In [299]:
ds.variables['latitude'][:]

masked_array(
  data=[[47.82437686, 47.82530568, 47.82623299, ..., 48.03540431,
         48.0349378 , 48.03446974],
        [47.83328709, 47.83421619, 47.83514379, ..., 48.04438033,
         48.04391368, 48.04344547],
        [47.84219727, 47.84312667, 47.84405455, ..., 48.05335634,
         48.05288954, 48.05242118],
        ...,
        [60.6577134 , 60.65920612, 60.66069645, ..., 60.99795737,
         60.99720274, 60.99644561],
        [60.66652742, 60.66802068, 60.66951154, ..., 61.00689465,
         61.00613976, 61.00538235],
        [60.67534134, 60.67683513, 60.67832653, ..., 61.01583191,
         61.01507673, 61.01431905]],
  mask=False,
  fill_value=1e+20)

In [298]:
ds.variables['projection_y_coordinate_bnds'][:][0, 0]

-200000.0

In [371]:
ds.variables['latitude'].shape, ds.variables['longitude'].shape

((1450, 900), (1450, 900))

In [431]:
1450*900

1305000

In [427]:
a = np.array([ds.variables['latitude'][:][400, 405], ds.variables['longitude'][:][400, 405]])

In [428]:
a

array([51.66909012, -4.81302932])

In [430]:
ds.variables['projection_x_coordinate'][:][405], ds.variables['projection_y_coordinate_bnds'][:][400]

(205500.0,
 masked_array(data=[200000., 201000.],
              mask=False,
        fill_value=1e+20))

In [424]:
from scipy.optimize import minimize
def objective(idx):
    i, j = idx
    print(idx)
    a = np.array([ds.variables['latitude'][:][i, j], ds.variables['longitude'][:][i, j]])
    print(a)
    b = np.array([47.824000, -10.0129000])
    return np.linalg.norm(a-b)

In [ ]:
res = minimize(objective, np.array([400, 405]), bounds = ((0, 1450), (0, 900)))

In [303]:
find_nearest(.flatten(), )

2.0136966587213836

In [304]:
np.argmin()

2699

In [355]:
A = ds.variables['latitude'][:]
np.unravel_index(np.abs(A-(51.5072)).argmin(), A.shape)

(405, 79)

In [356]:
A = ds.variables['longitude'][:]
np.unravel_index(np.abs(A-(-0.1276)).argmin(), A.shape)

(703, 721)

In [357]:
ds.variables['sfcWind'][:][0, 405, 79]

masked

In [358]:
ds.variables['sfcWind'][:][0, 703, 721]

masked

In [365]:
ds.variables['projection_y_coordinate_bnds'][:][405, :]

masked_array(data=[205000., 206000.],
             mask=False,
       fill_value=1e+20)

In [367]:
ds.variables['projection_y_coordinate'][:][405]

205500.0

In [324]:
ds.variables['sfcWind'][:][0, :].data[~ds.variables['sfcWind'][:][0, :].mask]

array([7.9356436 , 7.99015932, 8.02001338, ..., 9.92312173, 8.69825802,
       8.28347462])

In [328]:
A.shape

(1450, 900)

In [301]:
ds.variables['longitude'][:]

masked_array(
  data=[[-10.01291046,  -9.99967914,  -9.98644715, ...,   1.99162733,
           2.00501067,   2.01839366],
        [-10.01429114, -10.0010576 ,  -9.98782338, ...,   1.99232191,
           2.00570756,   2.01909287],
        [-10.01567249, -10.00243672,  -9.98920028, ...,   1.99301684,
           2.0064048 ,   2.01979242],
        ...,
        [-13.00292635, -12.98496742, -12.96700657, ...,   3.50519538,
           3.52359931,   3.54200224],
        [-13.00597103, -12.98800738, -12.97004181, ...,   3.50674729,
           3.52515634,   3.54356439],
        [-13.00901764, -12.99104927, -12.97307896, ...,   3.5083002 ,
           3.52671438,   3.54512755]],
  mask=False,
  fill_value=1e+20)

In [232]:
find_nearest(ds.variables['longitude'][:], -0.14159)

ValueError: object too deep for desired array

In [228]:
pd.tseries.offsets

<module 'pandas.tseries.offsets' from 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\pandas\\tseries\\offsets.py'>

In [210]:
ds.variables['longitude'][:][400:405, 400:405]

masked_array(
  data=[[-4.88524615, -4.87080337, -4.8563603 , -4.84191693, -4.82747327],
        [-4.88581728, -4.87137165, -4.85692573, -4.84247951, -4.828033  ],
        [-4.88638871, -4.87194023, -4.85749146, -4.84304239, -4.82859303],
        [-4.88696044, -4.87250911, -4.85805748, -4.84360556, -4.82915335],
        [-4.88753247, -4.87307828, -4.8586238 , -4.84416902, -4.82971396]],
  mask=False,
  fill_value=1e+20)

In [197]:
ds.variables

{'sfcWind': <class 'netCDF4._netCDF4.Variable'>
 float64 sfcWind(time, projection_y_coordinate, projection_x_coordinate)
     _FillValue: 1e+20
     standard_name: wind_speed
     long_name: Wind speed at 10m
     units: m s-1
     description: Wind speed
     label_units: m s-1
     level: 10m
     plot_label: Wind speed at 10m (m s-1)
     cell_methods: time: mean time: mean
     grid_mapping: transverse_mercator
     coordinates: latitude longitude
 unlimited dimensions: 
 current shape = (1, 1450, 900)
 filling on,
 'transverse_mercator': <class 'netCDF4._netCDF4.Variable'>
 int32 transverse_mercator()
     grid_mapping_name: transverse_mercator
     longitude_of_prime_meridian: 0.0
     semi_major_axis: 6377563.396
     semi_minor_axis: 6356256.909
     longitude_of_central_meridian: -2.0
     latitude_of_projection_origin: 49.0
     false_easting: 400000.0
     false_northing: -100000.0
     scale_factor_at_central_meridian: 0.9996012717
 unlimited dimensions: 
 current shape = (

In [196]:
ds['latitude'][:]

masked_array(
  data=[[47.82437686, 47.82530568, 47.82623299, ..., 48.03540431,
         48.0349378 , 48.03446974],
        [47.83328709, 47.83421619, 47.83514379, ..., 48.04438033,
         48.04391368, 48.04344547],
        [47.84219727, 47.84312667, 47.84405455, ..., 48.05335634,
         48.05288954, 48.05242118],
        ...,
        [60.6577134 , 60.65920612, 60.66069645, ..., 60.99795737,
         60.99720274, 60.99644561],
        [60.66652742, 60.66802068, 60.66951154, ..., 61.00689465,
         61.00613976, 61.00538235],
        [60.67534134, 60.67683513, 60.67832653, ..., 61.01583191,
         61.01507673, 61.01431905]],
  mask=False,
  fill_value=1e+20)